In [52]:
#import important libraries.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.api.types import is_numeric_dtype
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [53]:
#read dataset csv file.
df = pd.read_csv("Churn_Modelling.csv")

In [54]:
# find the features and observation in the data set.
df.shape

(10000, 14)

In [55]:
# find the datatypes of the features and target variable. 
df.dtypes

RowNumber            int64
CustomerId           int64
Surname             object
CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [56]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [57]:
dropped = ["RowNumber", "CustomerId", "Surname"]
for i in range(len(dropped)):
    del df[dropped[i]]
df.head()    

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [58]:
df["Balance"] = np.ceil(df["Balance"]).astype(int)
df["EstimatedSalary"] = np.ceil(df["EstimatedSalary"]).astype(int)

In [59]:
df = pd.get_dummies(df)
df.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0,1,1,1,101349,1,1,0,0,1,0
1,608,41,1,83808,1,0,1,112543,0,0,0,1,1,0
2,502,42,8,159661,3,1,0,113932,1,1,0,0,1,0
3,699,39,1,0,2,0,0,93827,0,1,0,0,1,0
4,850,43,2,125511,1,1,1,79085,0,0,0,1,1,0


In [60]:
def remove_outlier(feature):
            first_q = np.percentile(df[feature], 25)
            third_q = np.percentile(df[feature], 75)
            IQR = third_q - first_q
            IQR *= 1.5
            minimum = first_q - IQR
            maximum = third_q + IQR

            mean = df[feature].median()

            df.loc[df[feature] < minimum, feature] = mean
            df.loc[df[feature] > maximum, feature] = mean

outliers = ["CreditScore", "Age", "NumOfProducts"]
for i in range(len(outliers)):
    remove_outlier(outliers[i])
    

In [61]:
df.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619.0,42.0,2,0,1.0,1,1,101349,1,1,0,0,1,0
1,608.0,41.0,1,83808,1.0,0,1,112543,0,0,0,1,1,0
2,502.0,42.0,8,159661,3.0,1,0,113932,1,1,0,0,1,0
3,699.0,39.0,1,0,2.0,0,0,93827,0,1,0,0,1,0
4,850.0,43.0,2,125511,1.0,1,1,79085,0,0,0,1,1,0


In [62]:
X = df.iloc[:, df.columns != "Exited"]
y = df["Exited"]

In [63]:
#Using Python Code
class LinearTrainer:

    def __init__(self):

        # Learning Rate
        self.l_rate = 0.0001

        # Total iterations
        self.iterations = 20000

    def trains(self, x_data_train, y_data_train, theta_vector):

        x_data_train = np.column_stack((np.ones((x_data_train.shape[0], 1), dtype=float), x_data_train))
        for i in range(self.iterations):
            z = np.dot(x_data_train, theta_vector)
            sigmoid = (1 / (1 + np.exp(-z)))
            a = sigmoid - y_data_train
            temp = np.dot(x_data_train.T, a)
            temp = np.dot(temp, self.l_rate) / len(x_data_train)
            theta_vector = theta_vector - temp
        return theta_vector

    def classify(self, x_data_test, theta_vector):

        y_prediction = np.zeros((x_data_test.shape[0], 1), dtype=float)
        x_data_test = np.column_stack((np.ones((x_data_test.shape[0], 1)), x_data_test))
        z = np.dot(x_data_test, theta_vector)
        sigmoid = np.array(1 / (1 + np.exp(-z)))
        for i in (range(0, len(sigmoid))):
            if round(sigmoid[i][0], 2) <= 0.5:
                y_prediction[i][0] = 0
            else:
                y_prediction[i][0] = 1
        return y_prediction

    def accuracy(self, y_data_test, y_pred_test):
        total_error = 0
        for i in range(0, len(y_data_test)):
            total_error = total_error + abs((y_pred_test[i] - y_data_test[i]) / y_data_test[i])
        total_error = (total_error / len(y_data_test))
        accuracy = 1 - total_error
        return accuracy * 100

    def plotgraph(self, x_data_test, y_data_test, y_pred):
        plt.plot(x_data_test, y_data_test, 'or', label='whole data')
        plt.plot(x_data_test, y_pred, label='predicted value')
        plt.legend()
        plt.show()

def main():

    l_t = LinearTrainer()
    x_data = np.array(X, dtype=np.float128)
    y_data = np.array(y, dtype=np.float128)
    y_data = y_data.reshape(len(y_data), 1)

    x_data_train, x_data_test, y_data_train, y_data_test = train_test_split(x_data, y_data
                                                                            , test_size=0.3, random_state=0)

    x_data_train = (x_data_train - x_data_train.mean()) / x_data_train.std()

    theta_vector = np.zeros(((x_data_train.shape[1] + 1), 1), dtype='f')

    parameters = l_t.trains(x_data_train, y_data_train, theta_vector)
    y_prediction_test = l_t.classify(x_data_test, parameters)
    y_prediction_train = l_t.classify(x_data_train, parameters)

    train_acc = round(float(100 - np.mean(np.abs(y_prediction_train - y_data_train)) * 100))
    test_acc = round(float(100 - np.mean(np.abs(y_prediction_test - y_data_test)) * 100))

    print("Train accuracy:", train_acc)
    print("Test accuracy:", test_acc)


if __name__ == '__main__':
    main()

Train accuracy: 80
Test accuracy: 79


In [64]:
# Using sklearn linear regression model
model = LogisticRegression()
reg = model.fit(x_data_train,y_data_train)
reg.score(x_data_test, y_data_test)
y_pred =  reg.predict(x_data_test)
print("Train accuracy", accuracy_score(y_data_train, y_data_train) * 100)
print("Test accuracy", accuracy_score(y_data_test, y_pred) * 100)

Train accuracy 100.0
Test accuracy 79.4
